In [1]:
import json
import os
import pysolr
import requests
import pandas as pd

CORE_NAME = "IRF24P2"
VM_IP = "localhost"

In [2]:
def delete_core(core=CORE_NAME):
    print(os.system('sudo su - solr -c "/opt/solr/bin/solr delete -c {core}"'.format(core=core)))


def create_core(core=CORE_NAME):
    print(os.system(
        'sudo su - solr -c "/opt/solr/bin/solr create -c {core} -n data_driven_schema_configs"'.format(
            core=core)))

In [3]:
class Indexer:
    def __init__(self):
        self.solr_url = f'http://{VM_IP}:8983/solr/'
        self.connection = pysolr.Solr(self.solr_url + CORE_NAME, always_commit=True, timeout=5000000)

    def do_initial_setup(self):
        delete_core()
        create_core()

    def create_documents(self, docs):
        print(self.connection.add(docs))

    def add_fields(self):
          
        data = {
            "add-field": [
                {
                    "name": "revision_id",
                    "type": "string",
                    "indexed": True
    
                },
                {
                    "name": "title",
                    "type": "string",
                    "indexed": True
   
                },
                {
                    "name": "summary",
                    "type": "text_en",
   
                },
                {
                    "name": "url",
                    "type": "string",
    
                },
                {
                    "name": "topic",
                    "type": "string",
                    "multiValued": True
   
                }
               
            ]
        }

        
      
        print(self.solr_url + CORE_NAME + "/schema")
        print(requests.post(self.solr_url + CORE_NAME + "/schema", json=data).json())

In [4]:
# Setting up the core and adding the fields
i = Indexer()
i.do_initial_setup()
i.add_fields()


Deleting core 'IRF24P2' using CoreAdminRequest

256



ERROR: Error from server at http://localhost:8983/solr/admin/cores?action=UNLOAD&core=IRF24P2&indexInfo=true&deleteIndex=true&deleteDataDir=true&deleteInstanceDir=true&wt=json&version=2.2: null
request: GET

Error returned:
{
  "responseHeader":{
    "status":400,
    "QTime":10
  },
  "error":{
    "metadata":["error-class","org.apache.solr.common.SolrException","root-error-class","org.apache.solr.common.SolrException"],
    "msg":"Cannot unload non-existent core [IRF24P2]",
    "code":400
  }
}




Created new core 'IRF24P2'
0
http://localhost:8983/solr/IRF24P2/schema
{'responseHeader': {'status': 0, 'QTime': 1721}}


In [5]:
with open('wikipedia_combined.json', 'r') as f:
    wiki_json = json.load(f)

In [6]:
def parse_wiki_json(wiki_json):
    # Parse the JSON string if necessary
    if isinstance(wiki_json, str):
        wiki_json = json.loads(wiki_json)
    
    data = []
    for topic, entries in wiki_json.items():
        for entry in entries:
            data.append({
                'title': entry.get('title'),
                'revision_id': entry.get('revision_id'),
                'summary': entry.get('summary'),
                'url': entry.get('url'),
                'topic': topic
            })
    
    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
    return df

In [7]:
wikipedia_df = parse_wiki_json(wiki_json)

In [8]:
# Index the sample dataset
collection = wikipedia_df.to_dict('records')
i.create_documents(collection)

{
  "responseHeader":{
    "status":0,
    "QTime":27330
  }
}


In [9]:
import requests

# Solr core URL
solr_url = "http://34.44.184.192:8983/solr/IRF24P2"

# Get all fields in the schema
def fetch_schema_fields(solr_url):
    schema_fields_url = f"{solr_url}/schema/fields"
    response = requests.get(schema_fields_url)
    
    if response.status_code == 200:
        fields = response.json().get("fields", [])
        print("Fields in the schema:")
        for field in fields:
            print(f"Name: {field['name']}, Type: {field['type']}")
    else:
        print(f"Failed to fetch schema fields. Status Code: {response.status_code}, Message: {response.text}")

# Get all field types in the schema
def fetch_field_types(solr_url):
    schema_field_types_url = f"{solr_url}/schema/fieldtypes"
    response = requests.get(schema_field_types_url)
    
    if response.status_code == 200:
        field_types = response.json().get("fieldTypes", [])
        print("\nField Types in the schema:")
        for field_type in field_types:
            print(f"Name: {field_type['name']}, Class: {field_type['class']}")
    else:
        print(f"Failed to fetch field types. Status Code: {response.status_code}, Message: {response.text}")

# Execute the functions
fetch_schema_fields(solr_url)
fetch_field_types(solr_url)


Fields in the schema:
Name: _nest_path_, Type: _nest_path_
Name: _root_, Type: string
Name: _text_, Type: text_general
Name: _version_, Type: plong
Name: id, Type: string
Name: revision_id, Type: string
Name: summary, Type: text_en
Name: title, Type: string
Name: topic, Type: string
Name: url, Type: string

Field Types in the schema:
Name: _nest_path_, Class: solr.NestPathField
Name: ancestor_path, Class: solr.TextField
Name: binary, Class: solr.BinaryField
Name: boolean, Class: solr.BoolField
Name: booleans, Class: solr.BoolField
Name: delimited_payloads_float, Class: solr.TextField
Name: delimited_payloads_int, Class: solr.TextField
Name: delimited_payloads_string, Class: solr.TextField
Name: descendent_path, Class: solr.TextField
Name: ignored, Class: solr.StrField
Name: location, Class: solr.LatLonPointSpatialField
Name: location_rpt, Class: solr.SpatialRecursivePrefixTreeFieldType
Name: lowercase, Class: solr.TextField
Name: pdate, Class: solr.DatePointField
Name: pdates, Class: s

In [17]:
import pysolr

# Initialize the Solr client
solr_url = "http://34.44.184.192:8983/solr/IRF24P1"
solr = pysolr.Solr(solr_url, always_commit=True, timeout=10)

# Define query parameters
query = "health"  # Replace with your query term
topics = ["Health", "Education"]  # Replace with the desired topic(s)
k = 10  # Number of top results to retrieve

# Build the Solr query
fq_topics = " OR ".join([f'topic:"{topic}"' for topic in topics])  # Filter by topics
solr_query = {
    "q": f'summary:"{query}"',  # Query in title or summary
    "fq": fq_topics,  # Filter query by topics
    "fl": "id,revision_id,title,summary,URL,topic",  # Fields to retrieve
    "rows": 5,  # Number of results to return
    "sort": "score desc"  # Sort by relevance (default Solr behavior)
}

# Execute the query
results = solr.search(**solr_query)

# Process and display results
for result in results.docs:
  
    print(f"ID: {result['id']}, Title: {result['topic']}, Summary: {result['summary']}, URL: {result['url']}")


KeyError: 'url'

In [21]:
results.topic

AttributeError: 'Results' object has no attribute 'topic'